In [1]:
import requests
import math
import json
import pickle
import pandas as pd

# 0. publications

In [2]:
url_publications = "https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/publications/search"
query = '"global health"|"santé mondiale"'
params = { 
            'pageSize': 2000,
            'query': query,
            'sourceFields': ['id']
}

scanr_outputs = requests.post(url_publications, json=params).json()
print(scanr_outputs['total'])

1160


In [2]:
def get_publi(pub_id):
    new_id = pub_id.replace('/', "%252f")
    url = f"https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/publications/{new_id}"
    return requests.get(url).json()

def get_entity(ent_id):
    url = f"https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/structure/{ent_id}"
    return requests.get(url).json()

affiliation_labels = pickle.load(open('affiliation_labels.pkl', 'rb'))
def get_label(ent_id):
    if ent_id in affiliation_labels:
        return affiliation_labels[ent_id]
    if ent_id is None:
        return ''
    try:
        elt = get_entity(ent_id)
    except:
        elt = {}
        
    try:
        label = elt['label']['default']
    except:
        label = ''
        
    try:
        acronym = elt['acronym']['default']
    except:
        acronym = ''
    
    try:
        city = elt['address'][0]['city']
    except:
        city = ''
    
    res = f'{label} {acronym} {city}'
    affiliation_labels[ent_id] = res
    return res
        

In [4]:
publications = []
for ix, r in enumerate(scanr_outputs['results']):
    if ix%25 == 0:
        print(ix, end=',')
    elt = {'id': r['value']['id'], 'highlights':[h['type'] for h in r['highlights']]}
    publi = get_publi(r['value']['id'])
    elt.update(publi)
    publications.append(elt)

0,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,525,550,575,600,625,650,675,700,725,750,775,800,825,850,875,900,925,950,975,1000,1025,1050,1075,1100,1125,1150,

In [54]:

flat_list = [item for sublist in df_publications.highlights.tolist() for item in sublist]
pd.DataFrame({'t': flat_list}).t.value_counts()

source.title                  541
summary.default               433
title.default                 171
alternativeSummary.default     43
affiliations.label.en          28
affiliations.label.default     28
source.publisher               26
domains.label.default           4
domains.label.fr                4
authors.fullName                2
summary.en                      1
Name: t, dtype: int64

In [5]:
pickle.dump(publications, open('publications_global_health.pkl', 'wb'))

# from publications

In [3]:
publications = pickle.load(open('publications_global_health.pkl', 'rb'))
df_publications = pd.DataFrame(publications)

In [12]:
df_publications

,id,highlights,type,productionType,title,authors,authorsCount,externalIds,isOa,publicationDate,...,affiliations,lastUpdated,createdAt,doiUrl,submissionDate,inventionKind,inpadocFamily,isInternational,isOeb,patents
0,halshs-01063721,"[summary.default, title.default]",other,publication,{'default': 'Local Biologies and the Chemical ...,"[{'role': 'author', 'firstName': 'Alex', 'last...",1.0,"[{'id': 'halshs-01063721', 'type': 'halId_s'},...",True,1402876800000,...,"[{'id': '775664105', 'kind': ['Secteur public'...",1640887448652,1.571867e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hal-02906960,[title.default],report,publication,{'default': 'LA CHINE ET LA CRISE SANITAIRE DE...,"[{'person': {'id': 'idref077449959', 'firstNam...",1.0,"[{'id': 'hal-02906960', 'type': 'halId_s'}, {'...",True,1588291200000,...,"[{'id': '493902662', 'kind': ['Structure de re...",1640887827654,1.596098e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,inserm-03014802,[title.default],other,publication,{'default': 'POUR QUELLE RÉDUCTION DES RISQUES...,"[{'role': 'author', 'firstName': 'C', 'lastNam...",1.0,"[{'id': 'inserm-03014802', 'type': 'halId_s'},...",False,1561939200000,...,"[{'id': '200817366W', 'kind': ['Structure de r...",1640887887196,1.606370e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,these2016IEPP0024,"[summary.default, title.default, alternativeSu...",these,thesis,{'default': 'Reimagining international law to ...,"[{'person': {'id': 'idref199280908', 'firstNam...",8.0,"[{'id': '2016IEPP0024', 'type': 'scanr'}]",True,1472515200000,...,"[{'id': '197534316', 'kind': ['Secteur public'...",1640887183309,1.557355e+12,http://www.theses.fr/2016IEPP0024,NaN,NaN,NaN,NaN,NaN,NaN
4,doi10.7202/1044621ar,[summary.default],journal-article,publication,{'default': 'Des enjeux éthiques liés à l’util...,"[{'person': {'id': 'idref115707840', 'firstNam...",1.0,"[{'id': '10.7202/1044621ar', 'type': 'scanr'}]",True,1523491200000,...,NaN,1640886846749,1.549907e+12,http://doi.org/10.7202/1044621ar,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,halshs-01375922,[source.title],proceedings-article,publication,{'default': 'Between financial capitalism and ...,"[{'person': {'id': 'idref159502837', 'firstNam...",1.0,"[{'id': 'halshs-01375922', 'type': 'halId_s'},...",True,1466035200000,...,"[{'id': '200217606H', 'kind': ['Structure de r...",1640887447543,1.571860e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,hal-02015689,[source.title],proceedings-article,publication,{'default': 'Le rôle et la place du patient'},"[{'person': {'id': 'idref050490230', 'firstNam...",1.0,"[{'id': 'hal-02015689', 'type': 'halId_s'}, {'...",False,1513641600000,...,"[{'id': '201521833W', 'kind': ['Structure de r...",1640887730861,1.583147e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1157,doi10.1038/nature14132,"[affiliations.label.default, affiliations.labe...",journal-article,publication,{'default': 'New genetic loci link adipose and...,"[{'role': 'author', 'firstName': 'Dmitry', 'la...",569.0,"[{'id': 'hal-01132692', 'type': 'halId_s'}, {'...",True,1423612800000,...,"[{'id': '130015332', 'kind': ['Secteur public'...",1640886595237,1.549428e+12,http://doi.org/10.1038/nature14132,NaN,NaN,NaN,NaN,NaN,NaN
1158,hal-03296369,[source.title],book-chapter,publication,{'default': 'Les migrants sont par nature vuln...,"[{'person': {'id': 'idref133228592', 'firstNam...",1.0,"[{'id': 'hal-03296369', 'type': 'halId_s'}, {'...",False,1420070400000,...,"[{'id': '199812890Z', 'kind': ['Structure de r...",1640888042158,1.627534e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
authors = {}
idrefs = {}
affiliations = {}
for p in publications:
    keep = False
    for f in ['title.default', 'summary.default', 'alternativeSummary.default', 
              'domains.label.default', 'summary.en', 'domains.label.fr' ]:
        if f in p['highlights']:
            keep= True
    if keep is False:
        continue
    if isinstance(p.get('authors'), list):
        for aut in p['authors']:
            if aut:
                if aut['fullName'] not in authors:
                    authors[aut['fullName']] = {'publications': [], 'theses':[], 'fullName': aut['fullName']}
                if p['productionType'] == 'publication':
                    authors[aut['fullName']]['publications'].append(p)
                elif p['productionType'] == 'thesis':
                    authors[aut['fullName']]['theses'].append(p)
                
            if aut.get('person'):
                idref = aut['person']['id']
                if idref not in idrefs:
                    last_aff_id = None
                    if aut['person'].get('affiliations'):
                        max_endDate = max([d['endDate'] for d in aut['person']['affiliations']])
                        last_aff_id = [d for d in aut['person']['affiliations'] if d['endDate'] == max_endDate][0]['structure']['id']
                    idrefs[idref] = { 
                                     'idref': idref, 
                                     'fullName':aut['person'].get('fullName'),
                        'fullNameIdref':f"{aut['person'].get('fullName')}({idref})",
                                    'last_affiliation_id': last_aff_id,
                                    'last_affiliation': get_label(last_aff_id),
                                    'nb_publications': 0,
                                    'publications': [],
                                    'theses': []
                    }
                if p['productionType'] == 'publication': 
                    idrefs[idref]['publications'].append(p['id'])
                elif p['productionType'] == 'thesis':
                    idrefs[idref]['theses'].append(p['id'])
                
    if isinstance(p.get('affiliations'), list):
        for aff in p['affiliations']:
            if aff:
                key = aff['label']['default']+"###"+aff['id']
                if key not in affiliations:
                    affiliations[key] = {'publications': [], 'name': aff['label']['default'], 'id': aff['id']}
                affiliations[key]['publications'].append(p)

In [63]:
# https://scholar.google.com/citations?view_op=search_authors&hl=fr&mauthors=label:global_health
#211 -> eloi marijon

In [64]:
for aut in authors:
    authors[aut]['nb_publications'] = len(authors[aut]['publications'])
df_authors = pd.DataFrame(authors.values()).sort_values(by='nb_publications', ascending=False)
    
for idref in idrefs:
    idrefs[idref]['nb_publications'] = len(idrefs[idref]['publications'])
    idrefs[idref]['nb_theses'] = len(idrefs[idref]['theses'])
df_idrefs = pd.DataFrame(idrefs.values()).sort_values(by='nb_publications', ascending=False)
    
for aff in affiliations:
    affiliations[aff]['nb_publications'] = len(affiliations[aff]['publications'])
df_affiliations = pd.DataFrame(affiliations.values()).sort_values(by='nb_publications', ascending=False)

In [65]:
pickle.dump(df_idrefs, open('df_idrefs_global_health.pkl', 'wb'))

In [66]:
df_idrefs

,idref,fullName,fullNameIdref,last_affiliation_id,last_affiliation,nb_publications,publications,theses,nb_theses
6,idref115707840,Valéry Ridde,Valéry Ridde(idref115707840),200817718D,Centre population et développement CEPED Paris,18,"[doi10.7202/1044621ar, doi10.3917/ehesp.ridde....",[],0
8,idref057764468,Dominique Kerouedan,Dominique Kerouedan(idref057764468),180092165,Académie nationale de médecine AC N MEDECINE P...,7,"[doi10.4000/books.cdf.2288, doi10.4000/lettre-...",[],0
658,idref075743434,Tamara Giles-Vernick,Tamara Giles-Vernick(idref075743434),200118132J,Unité de Recherche et d'Expertise Epidémiologi...,5,"[doi10.1007/s10393-017-1283-4, doi10.4269/ajtm...",[],0
76,idref060719761,Frédéric Le Marcis,Frédéric Le Marcis(idref060719761),200511876S,"Triangle. Action, discours, pensée politique e...",4,"[halshs-03190368, halshs-01379211, halshs-0319...",[],0
13,idref116679085,Fatoumata Ouattara,Fatoumata Ouattara(idref116679085),200217423J,Laboratoire Population-Environnement-Développe...,4,"[hal-01473026, doi10.3917/ehesp.ridde.2015.01,...",[],0
...,...,...,...,...,...,...,...,...,...
643,idref069680329,Olivier Oullier,Olivier Oullier(idref069680329),201220328Y,Laboratoire de psychologie cognitive LPC Marse...,0,[],[these2017AIXM0018],1
642,idref225898675,Alice Soriano,Alice Soriano(idref225898675),130015332,Aix-Marseille University Aix-Marseille Univers...,0,[],[these2017AIXM0018],1
641,idref175799466,Yannick Fleury,Yannick Fleury(idref175799466),200810872M,Institut Universitaire Européen de la Mer IUEM...,0,[],[these2016BRES0127],1
640,idref192590596,Gaëtan Burgaud,Gaëtan Burgaud(idref192590596),200415142W,Laboratoire universitaire de biodiversité et é...,0,[],[these2016BRES0127],1


In [67]:
df_idrefs.to_csv('personnes_global_health.csv', index=False)

In [78]:
df_idrefs_main = df_idrefs[df_idrefs.nb_publications>1]
df_grp_struct1 = pd.DataFrame(df_idrefs_main.groupby(['last_affiliation', 'last_affiliation_id']).nb_publications.sum()).sort_values(by='nb_publications', ascending=False)
df_grp_struct2 = pd.DataFrame(df_idrefs_main.groupby(['last_affiliation', 'last_affiliation_id']).fullNameIdref.apply(list))
df_final = pd.merge(df_grp_struct1, df_grp_struct2, on=['last_affiliation_id', 'last_affiliation'])

# Structures
## webcontent

In [79]:
url_structures = "https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/search"
#my_query = '"' + '\"|\"'.join(keywords) + '"'

def get_prop(query, type_partenaire, searchFields):

    print(query)

    # list of the fields returned by the API
    sourceFields = ["id", "label.default"]
    
    
    filters = {'status': {'type': 'MultiValueSearchFilter',
       'op': 'all',
       'values': ['active']}
              }
    
    if type_partenaire == "fr":
        filters['isFrench'] = {'type': 'MultiValueSearchFilter',
       'op': 'all',
       'values': [True]}
   
        
    params = { 
            'pageSize': 2300,
            'query': query,
            'sourceFields': sourceFields,
            'searchFields': searchFields,
            'filters': filters}


    scanr_outputs = requests.post(url_structures, json=params).json()
    print("{} resultats".format(scanr_outputs.get('total')))
    return [e['value']['id'] for e in scanr_outputs.get('results')]

In [80]:
custom_searchFields = [
  #      "publications.publication.title.default",
  #      "publications.publication.keywords.default",
  #      "projects.project.label.en", "projects.project.label.default",
        "websites.webPages.content"
      #  "projects.project.keywords", "projects.project.keywords.default"
                   ]

searchFields = custom_searchFields

type_partenaire = ""

webcontent = {}
for query in ['"global health"', '"santé mondiale"']:
    webcontent[query] = get_prop(query, 'fr', searchFields)


"global health"
525 resultats
"santé mondiale"
122 resultats


In [81]:
for i, row in df_final.iterrows():
    current_webcontent=''
    if i[0] in webcontent['"global health"']:
        current_webcontent += 'global health'
    if i[0] in webcontent['"santé mondiale"']:
        current_webcontent += ';santé mondiale'
    df_final.at[i, 'webcontent'] = current_webcontent
df_final

,,nb_publications,fullNameIdref,webcontent
last_affiliation_id,last_affiliation,,,
200817718D,Centre population et développement CEPED Paris,23,"[Valéry Ridde(idref115707840), Fred Eboko(idre...",global health;santé mondiale
180092165,Académie nationale de médecine AC N MEDECINE Paris,7,[Dominique Kerouedan(idref057764468)],global health;santé mondiale
200217606H,"Centre de recherche médecine, sciences, santé, santé mentale, société CERMES 3 Villejuif",7,"[Claire Beaudevin(idref142978299), Laurent Por...",global health;santé mondiale
201420917E,Institut Pierre Louis d'épidémiologie et de santé publique iPLESP Paris,6,"[Laure Gossec(idref061096504), Florence Tubach...",
200217423J,Laboratoire Population-Environnement-Développement LPED Marseille,6,"[Fatoumata Ouattara(idref116679085), Fabienne ...",
...,...,...,...,...
200012689X,Laboratoire de cristallographie et rmn biologiques Paris,2,[Sylvie Nonin-Lecomte(idref168843862)],
269401154,Centre Hospitalier Intercommunal de Créteil Centre Hospitalier Intercommunal de Créteil CHIC CHI Créteil Créteil,2,[Christos Chouaïd(idref059755172)],
200615303M,Laboratoire de psychopathologie et processus de santé LPPS Boulogne-Billancourt,2,[Susana Tereno(idref22778748X)],global health


# projets

In [82]:
url_projets = "https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/projects/search"
query = '"global health"|"santé mondiale"'
params = { 
            'pageSize': 2000,
            'query': query,
            'sourceFields': ['id']
}

scanr_outputs = requests.post(url_projets, json=params).json()
print(scanr_outputs['total'])

247


In [83]:
def get_project(proj_id):
    r = requests.get(f'https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/projects/{proj_id}')
    return r.json()

In [84]:
projects = []
for ix, r in enumerate(scanr_outputs['results']):
    if ix%25 == 0:
        print(ix, end=',')
    elt = {'id': r['value']['id'], 'highlights':[h['type'] for h in r['highlights']]}
    project = get_project(r['value']['id'])
    elt.update(project)
    projects.append(elt)

0,25,50,75,100,125,150,175,200,225,

In [85]:
projects_by_struct = {}
for p in projects:
    for aff in p['participants']:
        if aff.get('structure') and aff['structure'].get('id'):
            struct_id = aff['structure'].get('id')
            if struct_id not in projects_by_struct:
                projects_by_struct[struct_id] = {'nb_projets': 0, 'ANR': 0, 'H2020': 0, 'projects':[] }
             
            projects_by_struct[struct_id]['projects'].append(p['id'])
            projects_by_struct[struct_id]['nb_projets'] += 1
            if p['type'] == 'ANR':
                projects_by_struct[struct_id]['ANR'] += 1
            if p['type'] in ['H2020', 'FP7']:
                projects_by_struct[struct_id]['H2020'] += 1
len(projects_by_struct)

899

In [86]:
df_final['projects']=''
for i, row in df_final.iterrows():
    nb_projets, ANR, H2020 = 0, 0, 0
    current_projects = []
    if i[0] in projects_by_struct:
        nb_projets = projects_by_struct[i[0]]['nb_projets']
        ANR = projects_by_struct[i[0]]['ANR']
        H2020 = projects_by_struct[i[0]]['H2020']
        current_projects = projects_by_struct[i[0]]['projects']
    df_final.at[i, 'nb_projets'] = nb_projets
    df_final.at[i, 'ANR'] = ANR
    df_final.at[i, 'H2020'] = H2020
    df_final.at[i, 'projects'] = current_projects
        
        
pickle.dump(df_final, open('df_final.global_health.pkl', 'wb'))

In [88]:
df_final.to_csv('structures_global_health.csv')